In [8]:
import pandas as pd
from collections import Counter
from itertools import chain
import datetime

# 데이터 불러오기
df = pd.read_csv("C:/Users/hp/Desktop/Bootcamp/PROJECT_OTT_AARRR/keyword_analysis/video_data_with_keywords_kobert.csv")

# 날짜만 추출 (datetime으로 변환 → 날짜만 사용)
df["date"] = pd.to_datetime(df["uploadDate"]).dt.date

# 날짜별 키워드 카운팅
daily_keyword_counts = (
    df.explode("keywords")                  # 키워드 리스트 → 여러 행으로
      .groupby(["date", "keywords"])       # 날짜 + 키워드 조합
      .size()
      .reset_index(name="count")           # 카운트 값
)

# 기준일: 가장 마지막 날짜 기준
max_date = df["date"].max()
recent_3_days = [max_date - datetime.timedelta(days=i) for i in range(3)]
prev_7_days = [max_date - datetime.timedelta(days=i) for i in range(3, 10)]

# 최근 3일, 이전 7일 빈도 집계
recent_counts = daily_keyword_counts[daily_keyword_counts["date"].isin(recent_3_days)]
prev_counts = daily_keyword_counts[daily_keyword_counts["date"].isin(prev_7_days)]

# 키워드별 합계
recent_total = recent_counts.groupby("keywords")["count"].sum()
prev_total = prev_counts.groupby("keywords")["count"].sum()

# 비율 계산 (최근 3일 / 이전 7일)
ratio_df = pd.DataFrame({
    "recent_3d": recent_total,
    "prev_7d": prev_total
}).fillna(0)

ratio_df["ratio"] = ratio_df["recent_3d"] / (ratio_df["prev_7d"] + 1e-5)  # 0 나누기 방지

# 반짝 키워드 기준: 비율이 2 이상 & 최근 등장 빈도 최소 3 이상
flash_keywords = ratio_df[(ratio_df["ratio"] >= 2.0) & (ratio_df["recent_3d"] >= 3)].sort_values("ratio", ascending=False)

# 각 키워드가 등장한 날짜 수
daily_count = daily_keyword_counts.groupby("keywords")["date"].nunique()

# 기준: 전체 날짜의 70% 이상 등장한 키워드 (꾸준한 키워드)
total_days = df["date"].nunique()
steady_keywords = daily_count[daily_count >= int(total_days * 0.7)].sort_values(ascending=False)

# 반짝 키워드
flash_keyword_list = flash_keywords.index.tolist()
# print(flash_keyword_list)
print(flash_keywords.head(20))

# 꾸준 키워드
steady_keyword_list = steady_keywords.index.tolist()
# print(steady_keyword_list)
print(steady_keywords.head(20))

                                                    recent_3d  prev_7d  \
keywords                                                                 
['ai 동요', '음악 아기노래', '동요', '아기노래', '동요 shorts']          20.0      0.0   
['실력 배구', '배구 short', '배구', '배구 코트에서', 'short 댄스']        4.0      1.0   
['2025년도 시청자에서애청자', 'view일정media schedule', 'vi...        3.0      1.0   

                                                           ratio  
keywords                                                          
['ai 동요', '음악 아기노래', '동요', '아기노래', '동요 shorts']     2.000000e+06  
['실력 배구', '배구 short', '배구', '배구 코트에서', 'short 댄스']  3.999960e+00  
['2025년도 시청자에서애청자', 'view일정media schedule', 'vi...  2.999970e+00  
keywords
['ai모델 ai화보', 'ai사진 ai모델', 'aivlog aigirl', 'ai실사 ai룩북', 'ai룩북 ai교복']    21
Name: date, dtype: int64
